**Imports**

In [1]:
import os
import dotenv
dotenv.load_dotenv(".env")

True

**Create Client**

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

**Knowledge Tools**

In [3]:
from azure.ai.projects.models import BingGroundingTool

bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"]
)
conn_id = bing_connection.id

print(conn_id)

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

/subscriptions/4ce7ad8d-95ed-4652-bd4a-5f2af19d29cb/resourceGroups/rg-ericsson-learning-path/providers/Microsoft.MachineLearningServices/workspaces/ericsson-learning-path-project/connections/bingericssonlearningpath


**Create Agent**

In [4]:
from azure.ai.projects.models import FunctionTool

agent = project_client.agents.create_agent(
    name="my-knowledge-tool-agent",
    model=os.getenv("chatModel"),
    instructions="You are a helpful assistant",
    tools=bing.definitions,
    # headers={"x-ms-enable-preview": "true"},
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_Bi7hfZ4Be1465NnU9W6IooY0


**Create Thread & Adding Message**

In [5]:
user_message = "hi, what's the weather in New York?"
# user_message = "Do a web search for latest news in AI"
user_message = "What is the latest news in AI when it comes to Google?"

In [6]:
# with project_client:
thread = project_client.agents.create_thread()
agent = project_client.agents.get_agent(agent_id=agent.id)

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_message,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_sqb0skR34FuQ4WsOn9VnjQs4


**Running Agent with Thread**

In [7]:
from azure.ai.projects.models import MessageRole

# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [9]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)

for m in reversed(messages.data):
    if m.content[0].text.annotations:
        print(f"Message ({m.role}): {m.content[0].text.value}")
        for ref in m.content[0].text.annotations:
            print(f"URL Citation: [{ref.url_citation.title}]({ref.url_citation.url})")
    else:
        print(f"Message ({m.role}): {m.content[0].text.value}")

Message (MessageRole.USER): What is the latest news in AI when it comes to Google?
Message (MessageRole.AGENT): The latest news in AI from Google involves the unveiling of Gemini 2.5, the company's most advanced AI model to date. This new AI reasoning model introduces a capability to pause and "think" before responding, significantly enhancing its accuracy and performance across various tasks, including coding and mathematics. Gemini 2.5 is available in the Gemini app for subscribers and on Google's developer platform, Google AI Studio【3†source】【0†source】.
URL Citation: [Google unveils a next-gen family of AI reasoning models](https://techcrunch.com/2025/03/25/google-unveils-a-next-gen-ai-reasoning-model/)
URL Citation: [Google Unveils Gemini 2.5, Its Most Advanced AI Model Yet: Here’s How It Works And What It Offers](https://in.mashable.com/tech/91750/google-unveils-gemini-25-its-most-advanced-ai-model-yet-heres-how-it-works-and-what-it-offers)


**Delete Agent & Thread**

In [10]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

# Delete Thread when done
project_client.agents.delete_thread(thread_id=thread.id)
print(f"thread: {thread.id} deleted")

Deleted agent
thread: thread_sSrf7iZwEx4xYLRCrCU3UI3e deleted
